In [1]:
# libraries
from selenium import webdriver
from time import sleep
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime

In [2]:
# links 
url = 'https://www.vivareal.com.br/aluguel/sp/sao-jose-dos-campos/apartamento_residencial/'

In [3]:
# abre o navegar  e entra na url de busca
driver = webdriver.Chrome()
driver.get(url)
sleep(4)
actions = ActionChains(driver)

In [4]:
# listas vazias dos resultados e para verificar se a url é repetida
listas = []
url_check = ['']
url_repeated = ['']

In [5]:
while True:
    # caso apareça o botão de cookies ele tentar clickar para aceitar.
    try:
        driver.find_element_by_class_name("cookie-notifier__cta").click()
    except:
        pass
    
    # aguarda 5 segundos para a página carregar
    sleep(5)
    
    # armazena a lista de resultados da busca
    result = driver.find_element_by_class_name("js-list")
    
    # lista das propriedades de cada cartão da lista de resultados
    list_apto = result.find_elements_by_class_name("js-property-card")
    
    # laço for para obter os dados de cada cartão de imóvel da lista
    for i,apto in enumerate(list_apto):
        
        # obtém o link URL do cartão que atual
        link = apto.find_element_by_class_name("js-card-title").get_attribute('href')
        
        # verificar se o link já foi utilizado na lista de urls armazenadas
        if link not in url_check:
            
            # comandas para obter as informações desejadas
            id_apto = link.split("id-")[-1][:-1]
            adress = apto.find_element_by_class_name("property-card__address").text
            area = apto.find_element_by_class_name("js-property-card-detail-area").text
            bedrooms = apto.find_element_by_class_name("js-property-detail-rooms").text
            bathrooms = apto.find_element_by_class_name("js-property-detail-bathroom").text
            garage = apto.find_element_by_class_name("js-property-detail-garages").text
            price = apto.find_element_by_class_name("js-property-card-prices").text
            try:
                others = apto.find_element_by_class_name("property-card__amenities").text
            except:
                others = None
            try:
                cond = apto.find_element_by_class_name("js-condo-price").text
            except:
                cond = None
                
            # armazena os resultados na lista de resultados 'listas' em forma de dicionário.
            listas.append({"id": id_apto,
                           "link": link,
                           "address": adress,
                           "area": area,
                           "bedrooms": bedrooms,
                           "bathrooms": bathrooms,
                           "garages": garage,
                           "others": others,
                           "price": price,
                           "condo": cond,
                           "date": datetime.now().strftime("%Y-%m-%d %H:%M")})
            url_check.append(link)
        else:
            url_repeated.append(link)
    
    # comando para avançar para próxima página
    paginator = driver.find_element_by_class_name("js-results-pagination")
    next_page = paginator.find_element_by_xpath("//a[@title='Próxima página']")
    # tenta clickar no botão próxima página, caso não for possível (última página) o laço while é quebrado com o break.
    try:
        next_page.click()
    except:
        break

# comando para fechar o navegador
driver.close()

In [6]:
# converte a lista em um DataFrame
df = pd.DataFrame(listas)

In [7]:
# extrai informações de texto
df['bedrooms'] = df['bedrooms'].str.extract('(.*) Quarto')

df['bathrooms'] = df['bathrooms'].str.extract('(.*) Banheiro')

df['garages'] = df['garages'].str.extract('(.*) Vaga')

df['price'] = df['price'].str.extract('R\$ (.*) \/mês.*')
df['price'] = df['price'].replace('\.','', regex=True)

df['condo'] = df['condo'].str.extract('R\$ (.*).*')
df['condo'] = df['condo'].replace('\.','', regex=True)

df['others'] = df['others'].replace('\n',', ', regex=True)

In [8]:
# cria coluna de bairro, as colunas que não contiverem , São José dos Campos - SP, serão preenchidas com nan.
df['neighborhood'] = df['address'].str.extract('(.*), São José dos Campos - SP')

In [9]:
# remove as linhas em que a coluna de bairro estiverem com nan.
df.drop(index=df[df['neighborhood'].isna()].index.values,inplace=True)
df.reset_index(inplace=True)

In [10]:
# filtra a string do bairro que contém os endereços de ruas/avenidas e pega somente o bairro
aux = len(df['neighborhood'])
count = 0
list_aux = []
while count < aux:
    if ' - ' in df['neighborhood'][count]:
        list_aux.append(df['neighborhood'][count].split(' - ')[-1])
    else:
        list_aux.append(df['neighborhood'][count])
    count+=1
df['neighborhood'] = list_aux

In [11]:
# seleciona colunas que serão transformadas em tipos numéricos
cols = ['id','area','bedrooms','bathrooms','garages','price','condo']
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

In [12]:
# converte a coluna em datetime
df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d %H:%M')

In [13]:
# coloca a coluna id como índice do DataFrame
df.set_index('id',inplace=True)

In [14]:
# elimina a coluna index
df.drop(columns = 'index',inplace=True)

In [15]:
df = df.fillna(0)

In [16]:
pd.DataFrame(df).to_csv("apto_results_final.csv", index=False,encoding='utf-8-sig')